- 기존 베이스라인을 바탕으로 pytorch 코드를 작성해 보았습니다. 
- 교차검증, datat augment 등 추가할 부분이 많은 코드입니다.
- 코드에 문제가 있거나 의문이 있으시면 댓글 남겨주세요 !! 


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings

warnings.filterwarnings("ignore")

## 데이터 불러오기

In [ ]:
sample_submission = pd.read_csv("./open/sample_submission.csv")

africa_train_paths = glob("./open/train/africa/*.wav")
australia_train_paths = glob("./open/train/australia/*.wav")
canada_train_paths = glob("./open/train/canada/*.wav")
england_train_paths = glob("./open/train/england/*.wav")
hongkong_train_paths = glob("./open/train/hongkong/*.wav")
us_train_paths = glob("./open/train/us/*.wav")

path_list = [africa_train_paths, australia_train_paths, canada_train_paths,
             england_train_paths, hongkong_train_paths, us_train_paths]

In [ ]:
# glob로 test data의 path를 불러올때 순서대로 로드되지 않을 경우를 주의해야 합니다.
# test_ 데이터 프레임을 만들어서 나중에 sample_submission과 id를 기준으로 merge시킬 준비를 합니다.

def get_id(data):
    return np.int(data.split("\\")[1].split(".")[0])

test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = glob("./open/test/*.wav")
test_["id"] = test_["path"].apply(lambda x : get_id(x))

test_.head()

## 데이터 전처리


In [ ]:
def load_data(paths):

    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미하는 것은 1초당 16000개의 데이터를 샘플링 한다는 것입니다.
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result) 
    # 메모리가 부족할 때는 데이터 타입을 변경해 주세요 ex) np.array(data, dtype = np.float32)

    return result

In [ ]:
# train 데이터를 로드하기 위해서는 많은 시간이 소모 됩니다.
# 따라서 추출된 정보를 npy파일로 저장하여 필요 할 때마다 불러올 수 있게 준비합니다.

os.mkdir("./npy_data")

africa_train_data = load_data(africa_train_paths)
np.save("./npy_data/africa_npy", africa_train_data)

australia_train_data = load_data(australia_train_paths)
np.save("./npy_data/australia_npy", australia_train_data)

canada_train_data = load_data(canada_train_paths)
np.save("./npy_data/canada_npy", canada_train_data)

england_train_data = load_data(england_train_paths)
np.save("./npy_data/england_npy", england_train_data)

hongkong_train_data = load_data(hongkong_train_paths)
np.save("./npy_data/hongkong_npy", hongkong_train_data)

us_train_data = load_data(us_train_paths)
np.save("./npy_data/us_npy", us_train_data)

test_data = load_data(test_["path"])
np.save("./npy_data/test_npy", test_data)

In [ ]:
# npy파일로 저장된 데이터를 불러옵니다.
africa_train_data = np.load("./npy_data/africa_npy.npy", allow_pickle = True)
australia_train_data = np.load("./npy_data/australia_npy.npy", allow_pickle = True)
canada_train_data = np.load("./npy_data/canada_npy.npy", allow_pickle = True)
england_train_data = np.load("./npy_data/england_npy.npy", allow_pickle = True)
hongkong_train_data = np.load("./npy_data/hongkong_npy.npy", allow_pickle = True)
us_train_data = np.load("./npy_data/us_npy.npy", allow_pickle = True)

test_data = np.load("./npy_data/test_npy.npy", allow_pickle = True)

train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

In [ ]:
# 이번 대회에서 음성은 각각 다른 길이를 갖고 있습니다.
# baseline 코드에서는 음성 중 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 사용합니다.

def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

#음성들의 길이를 맞춰줍니다.

def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

#feature를 생성합니다.

def get_feature(data, sr = 16000, n_fft = 256, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in data:
        # win_length 는 음성을 작은 조각으로 자를때 작은 조각의 크기입니다.
        # hop_length 는 음성을 작은 조각으로 자를때 자르는 간격을 의미합니다.
        # n_mels 는 적용할 mel filter의 개수입니다.
        mel_ = librosa.feature.melspectrogram(i, sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)

    mel_mean = mel.mean()
    mel_std = mel.std()
    mel = (mel - mel_mean) / mel_std

    return mel

In [ ]:
train_x = np.concatenate(train_data_list, axis= 0)
test_x = np.array(test_data)

# 음성의 길이 중 가장 작은 길이를 구합니다.

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

mini = np.min([train_mini, test_mini])

# data의 길이를 가장 작은 길이에 맞춰 잘라줍니다.

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

# librosa를 이용해 feature를 추출합니다.

train_x = get_feature(data = train_x)
test_x = get_feature(data = test_x)

train_x = train_x.reshape(-1, train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(-1, test_x.shape[1], test_x.shape[2], 1)

In [ ]:
train_y = np.concatenate((np.zeros(len(africa_train_data), dtype = np.int),
                        np.ones(len(australia_train_data), dtype = np.int),
                         np.ones(len(canada_train_data), dtype = np.int) * 2,
                         np.ones(len(england_train_data), dtype = np.int) * 3,
                         np.ones(len(hongkong_train_data), dtype = np.int) * 4,
                         np.ones(len(us_train_data), dtype = np.int) * 5), axis = 0)

In [ ]:
train_y = pd.get_dummies(train_y).to_numpy(dtype = 'long')

## 모델링


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms

### customDataset

In [ ]:
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        x, y = sample['x'], sample['y']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        x = x.transpose((2, 0, 1))
        return {'x': torch.FloatTensor(x),
                'y': torch.FloatTensor(y)}
to_tensor = transforms.Compose([
                      ToTensor() 
])
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,train_x,train_y,transforms=to_tensor):
        self.x_data = train_x
        self.y_data = train_y
        self.transforms = transforms# Transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx): 
        
        x = self.x_data[idx]
        y = self.y_data[idx]
        sample = {'x': x, 'y': y}
        if self.transforms:
            sample = self.transforms(sample)
        y = y.astype(np.float32)
        
        
        return (x,y)

In [ ]:
class MultiLabelResnet(nn.Module):
    def __init__(self):
        super(MultiLabelResnet, self).__init__()
        self.conv2d = nn.Conv2d(64, 3, 1, stride=1)
        self.resnet = timm.create_model('resnet101', pretrained=False) 
        self.FC = nn.Linear(1000, 6)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.relu(self.conv2d(x))

        # resnet18을 추가
        x = F.relu(self.resnet(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        #x = self.FC(x)
        x = torch.sigmoid(self.FC(x))
        return x
# 모델 선언

model = MultiLabelResnet()
model.to(device)

In [ ]:
dataset = CustomDataset(train_x,train_y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion  = torch.nn.BCELoss()

In [ ]:
from tqdm import tqdm

# 모델의 dropoupt, batchnormalization를 train 모드로 설정
model.train()

for epoch in range(10):
    # 1개 epoch 훈련
    train_acc_list = []
    with tqdm(dataloader,#train_data_loader를 iterative하게 반환
            total=dataloader.__len__(), # train_data_loader의 크기
            unit="batch") as train_bar: # 한번 반환하는 smaple의 단위는 "batch"
        for idx,sample in enumerate(train_bar):
            if idx == 224 :
                break
            train_bar.set_description(f"Train Epoch {epoch}")
            # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
            # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
            optimizer.zero_grad()

            images, labels = sample
            # tensor를 gpu에 올리기 
            images = images.to(device)
            labels = labels.to(device)


            
            # .forward()에서 중간 노드의 gradient를 계산
            with torch.set_grad_enabled(True):
                # 모델 예측
                probs = model(images)
                probs = F.softmax(probs)
               # probs = (probs == probs.max()) * 1.0
                #loss = criterion(probs, y_train)
                
                
                loss = criterion(probs, labels)
                #loss = criterion(probs, torch.max(y_train, 1)[1])

                loss.backward()
                optimizer.step()
                
                probs  = probs.cpu().detach().numpy()
                labels = labels.cpu().detach().numpy()
                # train accuracy 계산
                cnt = 0
                for i in range(256):
                    
                    if probs[i].argmax() == labels[i].argmax():
                        cnt +=1
                
                #preds = probs > 0.5
                #batch_acc = (labels == preds).mean()
                batch_acc = cnt/256
                train_acc_list.append(batch_acc)
                train_acc = np.mean(train_acc_list)

            # 현재 progress bar에 현재 미니배치의 loss 결과 출력
            train_bar.set_postfix(train_loss= loss.item(),
                                    train_acc = train_acc)
            

Train Epoch 0:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.304, train_loss=0.446]  
Train Epoch 1:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.393, train_loss=0.443]  
Train Epoch 2:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.395, train_loss=0.441]  
Train Epoch 3:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.396, train_loss=0.436]  
Train Epoch 4:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.395, train_loss=0.436]  
Train Epoch 5:  90%|████████▉ | 224/250 [01:35<00:11,  2.33batch/s, train_acc=0.395, train_loss=0.432]  
Train Epoch 6:  90%|████████▉ | 224/250 [01:36<00:11,  2.33batch/s, train_acc=0.396, train_loss=0.43]   
Train Epoch 7:  90%|████████▉ | 224/250 [01:36<00:11,  2.33batch/s, train_acc=0.396, train_loss=0.429]  
Train Epoch 8:  90%|████████▉ | 224/250 [01:35<00:11,  2.34batch/s, train_acc=0.395, train_loss=0.423]  
Train Epoch 9:  90%|████████▉ | 224/250 [01:36<00:11,  2.33batch/s, train_acc=0.397, train_loss=0.424]  

## 예측

In [ ]:
model.eval()

In [ ]:
test_y = pd.DataFrame(index=range(0,len(test_x)), columns=['0', '1', '2', '3', '4', '5'])
test_y = test_y.fillna(0).to_numpy()
dataset = CustomDataset(test_x,test_y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

 

In [ ]:
pred_ = []
for idx, sample in enumerate(dataloader):
    with torch.no_grad():
        # 추론
        model.eval()
        images,_ = sample
        images = images.to(device)
        probs  = model(images)
        probs = F.softmax(probs)
        probs = probs.cpu().detach().numpy()
        pred_.append(probs)
        

In [ ]:
def cov_type(data):
    return np.int(data)

# 처음에 살펴본 것처럼 glob로 test data의 path는 sample_submission의 id와 같이 1,2,3,4,5.....으로 정렬 되어있지 않습니다.
# 만들어둔 test_ 데이터프레임을 이용하여 sample_submission과 predict값의 id를 맞춰줍니다.
sample_submission = pd.read_csv("./open/sample_submission.csv")
result = pd.concat([test_, pd.DataFrame(np.mean(pred, axis = 0))], axis = 1).iloc[:, 1:]
result["id"] = result["id"].apply(lambda x : cov_type(x))

result = pd.merge(sample_submission["id"], result)
result.columns = sample_submission.columns

In [ ]:
result.to_csv("DACON.csv", index = False)